In [1]:
import torch
from torch2trt import TRTModule
import torchvision.transforms as transforms
import torch.nn.functional as F
import cv2
import PIL.Image
import numpy as np
from IPython.display import display
import ipywidgets
import traitlets
from jetbot import Camera, bgr8_to_jpeg
from jetbot import Robot
import socket, json, time
import time


device = torch.device('cuda')

ModuleNotFoundError: No module named 'torch2trt'

Load the TRT optimized model by executing the cell below

In [3]:
model_trt = TRTModule()
model_trt.load_state_dict(torch.load('./models/trt_best_steering_model_xy_ResNet18.pth'))

NameError: name 'TRTModule' is not defined

In [3]:
mean = torch.Tensor([0.485, 0.456, 0.406]).cuda().half()
std = torch.Tensor([0.229, 0.224, 0.225]).cuda().half()

def preprocess(image):
    image = PIL.Image.fromarray(image)
    image = transforms.functional.to_tensor(image).to(device).half()
    image.sub_(mean[:, None, None]).div_(std[:, None, None])
    return image[None, ...]

display camera

In [2]:
camera = Camera()
robot = Robot()

image_widget = ipywidgets.Image()
traitlets.dlink((camera, 'value'), (image_widget, 'value'), transform=bgr8_to_jpeg)

display(image_widget)

SyntaxError: invalid syntax (3939321600.py, line 1)

1. Speed Control (speed_gain_slider): To start your JetBot increase ``speed_gain_slider`` 
2. Steering Gain Control (steering_gain_slider): If you see JetBot is wobbling, you need to reduce ``steering_gain_slider`` till it is smooth
3. Steering Bias control (steering_bias_slider): If you see JetBot is biased towards extreme right or extreme left side of the track

In [7]:
speed_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, description='speed gain')
steering_gain_slider = ipywidgets.FloatSlider(min=0.0, max=1.0, step=0.01, value=0.2, description='steering gain')
steering_dgain_slider = ipywidgets.FloatSlider(min=0.0, max=0.5, step=0.001, value=0.0, description='steering kd')
steering_bias_slider = ipywidgets.FloatSlider(min=-0.3, max=0.3, step=0.01, value=0.0, description='steering bias')

display(speed_gain_slider, steering_gain_slider, steering_dgain_slider, steering_bias_slider)

FloatSlider(value=0.0, description='speed gain', max=1.0, step=0.01)

FloatSlider(value=0.2, description='steering gain', max=1.0, step=0.01)

FloatSlider(value=0.0, description='steering kd', max=0.5, step=0.001)

FloatSlider(value=0.0, description='steering bias', max=0.3, min=-0.3, step=0.01)

The x and y sliders will display the predicted x, y values.

In [8]:
x_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='x')
y_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, orientation='vertical', description='y')
steering_slider = ipywidgets.FloatSlider(min=-1.0, max=1.0, description='steering')
speed_slider = ipywidgets.FloatSlider(min=0, max=1.0, orientation='vertical', description='speed')

display(ipywidgets.HBox([y_slider, speed_slider]))
display(x_slider, steering_slider)

FloatSlider(value=0.0, description='x', max=1.0, min=-1.0)

FloatSlider(value=0.0, description='steering', max=1.0, min=-1.0)

In [9]:
# ROS bridge via UDP 

# host-side udp_to_cmdvel_ai.py listens on this port
UDP_IP = "127.0.0.1" 
UDP_PORT = 5005


ENABLE_DRIVE = True
INVERT_STEERING = True
SEND_HZ = 30

_sock = socket.socket(socket.AF_INET, socket.SOCK_DGRAM)

def send_cmd(linear_x, angular_z):
    payload = {"linear_x": float(linear_x), "angular_z": float(angular_z)}
    _sock.sendto(json.dumps(payload).encode("utf-8"), (UDP_IP, UDP_PORT))

def safe_send(linear_x, angular_z):
    if ENABLE_DRIVE:
        send_cmd(linear_x, angular_z)
    else:
        send_cmd(0.0, 0.0)

In [10]:
angle = 0.0
angle_last = 0.0
point_px = (0, 0)
latest_linear_x = 0.0
latest_angular_z = 0.0

last_send_ts = 0.0
def execute(change):
    global angle, angle_last, point_px, latest_linear_x, latest_angular_z
    image = change['new']
    xy = model_trt(preprocess(image)).detach().float().cpu().numpy().flatten()    
    x = 2.0 * (xy[0] - 0.5)
    y = xy[1]
    
    pred_x_norm = xy[0]
    pred_y_norm = xy[1]
    pred_x = int(np.clip(pred_x_norm * 224, 0, 224 - 1))
    pred_y = int(np.clip(pred_y_norm * 224, 0, 224 - 1))
    
    point_px = (pred_x, pred_y)

    x_slider.value = x
    y_slider.value = y
    
    speed_slider.value = speed_gain_slider.value
    
    angle = np.arctan2(x, y)
    pid = angle * steering_gain_slider.value + (angle - angle_last) * steering_dgain_slider.value
    angle_last = angle
    
    steering_slider.value = pid + steering_bias_slider.value
    
    # Command output (Docker -> Host via UDP)
    # publish a Twist-like pair: linear_x and angular_z.
    # linear_x is mapped from speed_slider; angular_z from steering_slider.
    latest_linear_x = float(speed_slider.value)
    latest_angular_z = float(steering_slider.value)
    linear_x = latest_linear_x
    angular_z = latest_angular_z
    if INVERT_STEERING:
        angular_z = -angular_z
        
    now_ts = time.time()
    if now_ts - last_send_ts >= (1.0 / float(SEND_HZ)):
        send_cmd(linear_x, angular_z)
        globals()['last_send_ts'] = now_ts



In [12]:
def draw_point_and_convert(frame):
    frame = frame.copy()

    # Draw red point (BGR format)
    cv2.circle(
        frame,
        center=point_px,
        radius=5,
        color=(0, 0, 255),
        thickness=-1
    )

    return bgr8_to_jpeg(frame)

In [13]:
camera_link = traitlets.dlink(
    (camera, 'value'),
    (image_widget, 'value'),
    transform=draw_point_and_convert
)

In [11]:
execute({'new': camera.value})

In [12]:
camera.observe(execute, names='value')

In [13]:
camera.unobserve(execute, names='value')

time.sleep(0.1)  # add a small sleep to make sure frames have finished processing

robot.stop()

In [14]:
camera.stop()